# LCEL

This lesson introduces LangChain Expression Language - an easy way to compose different langchain components. 

In it, we will go over the syntax for composing things together, and then the different interfaces it exposes

In [18]:
!pip install pydantic==1.10.8

  Obtaining dependency information for pydantic==1.10.8 from https://files.pythonhosted.org/packages/6c/32/0755046e707a468fe276fd40df11d492a72d1cbcfa344091e3a46120131c/pydantic-1.10.8-cp310-cp310-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 6.4 MB/s eta 0:00:00
Using cached pydantic-1.10.8-cp310-cp310-macosx_11_0_arm64.whl (2.5 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.0
    Uninstalling pydantic-1.10.0:
      Successfully uninstalled pydantic-1.10.0


In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

In [2]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [3]:
chain = prompt | model | output_parser

In [4]:
chain.invoke({"topic": "bears"})

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [5]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.schema.runnable import RunnableMap

In [8]:
vectorstore = DocArrayInMemorySearch.from_texts(["harrison worked at kensho"], embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [9]:
chain = (
    RunnableMap({
        "context": lambda x: retriever.get_relevant_documents(x["question"]),
        "question": lambda x: x["question"],
    })
    | prompt 
    | model 
    | StrOutputParser()
)

In [10]:
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

## Bind

and OpenAI Functions

In [11]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [13]:
runnable = prompt | model

In [14]:
runnable.invoke({"input": "Whats the weather in SF?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'weather_search', 'arguments': '{\n  "airport_code": "SFO"\n}'}})

## Fallbacks

In [15]:
from langchain.schema.output_parser import StrOutputParser
from langchain.llms import OpenAI
import json

In [16]:
simple_model = OpenAI(temperature=0, max_tokens=1000, model="text-davinci-001")
simple_chain = simple_model | json.loads

In [17]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line, author birth date"

In [18]:
simple_model.invoke(challenge)

', and author death date\n\n["The Raven","Edgar Allan Poe","Once upon a midnight dreary, while I pondered, weak and weary"]\n\n["The Waste Land","T.S. Eliot","April is the cruelest month, breeding lilacs out of the dead land")\n\n["The Love Song of J. Alfred Prufrock","T.S. Eliot","Let us go then, you and I, when the evening is spread out against the sky like a patient etherized upon a table")\n\n1888-1965\n1888-1965'

In [19]:
simple_chain.invoke(challenge)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [ ]:
chain.invoke(challenge)

In [ ]:
final_chain = simple_chain.with_fallbacks([chain])

In [ ]:
final_chain.invoke(challenge)

## Interface

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

### Invoke - single input

In [14]:
chain.invoke({"topic": "bears"})

"Why don't bears wear shoes?\n\nBecause they have bear feet!"

### Batch - many inputs

In [15]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why don't bears wear shoes?\n\nBecause they have bear feet!",
 'Why did the frog take the bus to work?\n\nBecause his car got toad away!']

### Stream

In [16]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 don
't
 bears
 wear
 shoes
?
 


Because
 they
 have
 bear
 feet
!



### Async

All of these have async methods: `ainvoke`, `abatch`, `astream`

In [17]:
response = await chain.ainvoke({"topic": "bears"})
response

"Why don't bears wear shoes?\n\nBecause they already have bear feet!"